In [1]:
import numpy as np
# load data from ReachData.npz
data=np.load('/Users/yangrenqin/GitHub/HW5/ReachData.npz')
r=data['r']
targets=data['targets']
target_index=data['cfr']
data.close()   

In [2]:
targets

array([[-98, -17],
       [-86,  50],
       [-64, -76],
       [-34,  93],
       [ 34,  93],
       [ 64, -76],
       [ 86,  50],
       [ 98, -17]], dtype=int16)

In [3]:
import math
degrees=[]
for i in targets:
    degree=math.degrees(math.atan2(i[1],i[0]))
    if degree < 0:
        degree=360+degree
    degrees.append(degree)

In [4]:
degrees

[189.84113175963438,
 149.82647997035568,
 229.89909245378774,
 110.08197748418075,
 69.91802251581925,
 310.1009075462122,
 30.173520029644333,
 350.1588682403656]

In [5]:
import pandas as pd
import random

cfr=pd.Series(target_index)
training_data=np.array([])
testing_data=np.array([])
# randomly select 400 trials(50 trials for each target) as traning data, and also pick out remaining data as test data 
for i in range(8):
    i+=1
    cfr_i=cfr[cfr.values==i]
    t1=random.sample(range(len(cfr_i.index)),50)
    t1.sort()
    t2=[cfr_i.index[l] for l in t1]
    t3=list(set(cfr_i.index)-set(t2))
    training_data=np.append(training_data,t2)
    testing_data=np.append(testing_data,t3)

In [6]:
training_data.sort()
training_data=np.int_(training_data)
# calculate spikes in plan, move and combined window individually, and its respective time with all the 190 neurons.
N=[]
N_time=[]
n_plan=[]
n_plantime=[]
n_move=[]
n_movetime=[]
for i in range(len(training_data)):
    p1=r[training_data[i]].timeTouchHeld
    p2=r[training_data[i]].timeGoCue
    p3=r[training_data[i]].timeTargetAcquire
    N2,N_time2,n_plan2,n_plantime2,n_move2,n_movetime2=np.array([]),np.array([]),np.array([]),np.array([]),np.array([]),np.array([])
    for l in range(190):
        if type(r[training_data[i]].unit[l].spikeTimes) == float:   # when there is only one spike and its spiketime
            N0=(r[training_data[i]].unit[l].spikeTimes>p1) & (r[training_data[i]].unit[l].spikeTimes<p3)
            N1=np.sum(N0)
            n_plan0=(r[training_data[i]].unit[l].spikeTimes>p1) & (r[training_data[i]].unit[l].spikeTimes<p2)
            n_plan1=np.sum(n_plan0)
            n_move0=(r[training_data[i]].unit[l].spikeTimes>p2) & (r[training_data[i]].unit[l].spikeTimes<p3)
            n_move1=np.sum(n_move0)
        elif list(r[training_data[i]].unit[l].spikeTimes) == []:   # when there is no spike and its spiketime
            N1=0
            n_plan1=0
            n_move1=0
        else:                                                     # when there are more than one spikes and their spiketimes
            N0=(r[training_data[i]].unit[l].spikeTimes>p1) & (r[training_data[i]].unit[l].spikeTimes<p3)
            N1=np.sum(N0)
            n_plan0=(r[training_data[i]].unit[l].spikeTimes>p1) & (r[training_data[i]].unit[l].spikeTimes<p2)
            n_plan1=np.sum(n_plan0)
            n_move0=(r[training_data[i]].unit[l].spikeTimes>p2) & (r[training_data[i]].unit[l].spikeTimes<p3)
            n_move1=np.sum(n_move0)
        N_time1=p3-p1
        n_movetime1=p3-p2
        n_plantime1=p2-p1
        
        N2=np.append(N2,N1)
        
        n_plan2=np.append(n_plan2,n_plan1)
        
        n_move2=np.append(n_move2,n_move1)
        
    N.append(N2)
    N_time.append(N_time1)
    n_plan.append(n_plan2)
    n_plantime.append(n_plantime1)
    n_move.append(n_move2)
    n_movetime.append(n_movetime1)

In [7]:
target0=[cfr[i] for i in training_data]
table1=pd.DataFrame(target0,index=training_data,columns=['targets']) # index represent the i th trials
table1['Combined']=N
table1['Combined_time']=N_time
table1['n_plan']=n_plan
table1['n_plantime']=n_plantime
table1['n_move']=n_move
table1['n_movetime']=n_movetime
table1['combined_rate']=table1['Combined']/table1['Combined_time']
table1['plan_rate']=table1['n_plan']/table1['n_plantime']
table1['move_rate']=table1['n_move']/table1['n_movetime']
table1

,targets,Combined,Combined_time,n_plan,n_plantime,n_move,n_movetime,combined_rate,plan_rate,move_rate
0,4,"[3.0, 16.0, 4.0, 24.0, 30.0, 12.0, 15.0, 31.0,...",1525,"[3.0, 8.0, 1.0, 9.0, 14.0, 4.0, 5.0, 16.0, 4.0...",1005,"[0.0, 8.0, 3.0, 15.0, 16.0, 8.0, 10.0, 15.0, 9...",520,"[0.00196721311475, 0.0104918032787, 0.00262295...","[0.00298507462687, 0.00796019900498, 0.0009950...","[0.0, 0.0153846153846, 0.00576923076923, 0.028..."
2,7,"[1.0, 11.0, 3.0, 19.0, 26.0, 17.0, 13.0, 14.0,...",1495,"[1.0, 3.0, 2.0, 5.0, 10.0, 6.0, 5.0, 8.0, 6.0,...",1005,"[0.0, 8.0, 1.0, 14.0, 16.0, 11.0, 8.0, 6.0, 8....",490,"[0.00066889632107, 0.00735785953177, 0.0020066...","[0.000995024875622, 0.00298507462687, 0.001990...","[0.0, 0.0163265306122, 0.00204081632653, 0.028..."
9,1,"[5.0, 16.0, 2.0, 4.0, 19.0, 7.0, 4.0, 25.0, 12...",1600,"[5.0, 7.0, 1.0, 2.0, 10.0, 2.0, 0.0, 17.0, 6.0...",1005,"[0.0, 9.0, 1.0, 2.0, 9.0, 5.0, 4.0, 8.0, 6.0, ...",595,"[0.003125, 0.01, 0.00125, 0.0025, 0.011875, 0....","[0.00497512437811, 0.00696517412935, 0.0009950...","[0.0, 0.0151260504202, 0.00168067226891, 0.003..."
11,5,"[3.0, 15.0, 0.0, 21.0, 30.0, 17.0, 10.0, 10.0,...",1330,"[3.0, 4.0, 0.0, 6.0, 6.0, 6.0, 1.0, 5.0, 0.0, ...",755,"[0.0, 11.0, 0.0, 15.0, 24.0, 11.0, 9.0, 5.0, 9...",575,"[0.00225563909774, 0.0112781954887, 0.0, 0.015...","[0.00397350993377, 0.00529801324503, 0.0, 0.00...","[0.0, 0.0191304347826, 0.0, 0.0260869565217, 0..."
13,2,"[0.0, 18.0, 6.0, 16.0, 20.0, 18.0, 7.0, 25.0, ...",1525,"[0.0, 5.0, 4.0, 7.0, 10.0, 6.0, 0.0, 12.0, 6.0...",1005,"[0.0, 13.0, 2.0, 9.0, 10.0, 12.0, 7.0, 13.0, 7...",520,"[0.0, 0.0118032786885, 0.00393442622951, 0.010...","[0.0, 0.00497512437811, 0.00398009950249, 0.00...","[0.0, 0.025, 0.00384615384615, 0.0173076923077..."
16,7,"[0.0, 19.0, 4.0, 22.0, 17.0, 19.0, 14.0, 16.0,...",1500,"[0.0, 13.0, 3.0, 10.0, 11.0, 8.0, 6.0, 5.0, 6....",1005,"[0.0, 6.0, 1.0, 12.0, 6.0, 11.0, 8.0, 11.0, 6....",495,"[0.0, 0.0126666666667, 0.00266666666667, 0.014...","[0.0, 0.0129353233831, 0.00298507462687, 0.009...","[0.0, 0.0121212121212, 0.0020202020202, 0.0242..."
17,4,"[1.0, 18.0, 4.0, 12.0, 14.0, 12.0, 7.0, 16.0, ...",1320,"[1.0, 8.0, 2.0, 6.0, 7.0, 4.0, 2.0, 8.0, 4.0, ...",755,"[0.0, 10.0, 2.0, 6.0, 7.0, 8.0, 5.0, 8.0, 9.0,...",565,"[0.000757575757576, 0.0136363636364, 0.0030303...","[0.00132450331126, 0.0105960264901, 0.00264900...","[0.0, 0.0176991150442, 0.00353982300885, 0.010..."
23,6,"[0.0, 16.0, 0.0, 16.0, 19.0, 19.0, 10.0, 14.0,...",1195,"[0.0, 3.0, 0.0, 11.0, 8.0, 5.0, 3.0, 7.0, 4.0,...",755,"[0.0, 13.0, 0.0, 5.0, 11.0, 14.0, 7.0, 7.0, 8....",440,"[0.0, 0.0133891213389, 0.0, 0.0133891213389, 0...","[0.0, 0.00397350993377, 0.0, 0.0145695364238, ...","[0.0, 0.0295454545455, 0.0, 0.0113636363636, 0..."
24,8,"[0.0, 10.0, 1.0, 12.0, 18.0, 16.0, 14.0, 11.0,...",1200,"[0.0, 4.0, 0.0, 3.0, 6.0, 9.0, 7.0, 2.0, 3.0, ...",755,"[0.0, 6.0, 1.0, 9.0, 12.0, 7.0, 7.0, 9.0, 6.0,...",445,"[0.0, 0.00833333333333, 0.000833333333333, 0.0...","[0.0, 0.00529801324503, 0.0, 0.00397350993377,...","[0.0, 0.0134831460674, 0.00224719101124, 0.020..."
25,7,"[0.0, 17.0, 4.0, 19.0, 29.0, 14.0, 11.0, 11.0,...",1455,"[0.0, 7.0, 2.0, 11.0, 14.0, 4.0, 2.0, 7.0, 4.0...",1005,"[0.0, 10.0, 2.0, 8.0, 15.0, 10.0, 9.0, 4.0, 2....",450,"[0.0, 0.0116838487973, 0.00274914089347, 0.013...","[0.0, 0.00696517412935, 0.00199004975124, 0.01...","[0.0, 0.0222222222222, 0.00444444444444, 0.017..."


In [8]:
combined_mean=[]
combined_cov=[]

plan_mean=[]
plan_cov=[]

move_mean=[]
move_cov=[]

for i in range(8):
    i=i+1
    combined=np.array(list(table1[table1.targets==i]['combined_rate']))
    combined_mean1=np.mean(combined,axis=0)
        
    combined_mean.append(combined_mean1)
    combined_cov.append(np.cov(combined.T))
    
    plan=np.array(list(table1[table1.targets==i]['plan_rate']))
    plan_mean1=np.mean(plan,axis=0)

        
    plan_mean.append(plan_mean1)
    plan_cov.append(np.cov(plan.T))
    
    move=np.array(list(table1[table1.targets==i]['move_rate']))
    move_mean1=np.mean(move,axis=0)

        
    move_mean.append(move_mean1)
    move_cov.append(np.cov(move.T))

In [9]:
testing_data.sort()
testing_data=np.int_(testing_data)
test_N=[]
test_N_time=[]
test_n_plan=[]
test_n_plantime=[]
test_n_move=[]
test_n_movetime=[]
# calculate spikes in plan, move and combined window individually, and its respective time with all the 190 neurons.
for i in range(len(testing_data)):
    p1=r[testing_data[i]].timeTouchHeld
    p2=r[testing_data[i]].timeGoCue
    p3=r[testing_data[i]].timeTargetAcquire
    test_N2,test_N_time2,test_n_plan2,test_n_plantime2,test_n_move2,test_n_movetime2=np.array([]),np.array([]),np.array([]),np.array([]),np.array([]),np.array([])
    for l in range(190):
        if type(r[testing_data[i]].unit[l].spikeTimes) == float:
            test_N0=(r[testing_data[i]].unit[l].spikeTimes>p1) & (r[testing_data[i]].unit[l].spikeTimes<p3)
            test_N1=np.sum(test_N0)
            test_n_plan0=(r[testing_data[i]].unit[l].spikeTimes>p1) & (r[testing_data[i]].unit[l].spikeTimes<p2)
            test_n_plan1=np.sum(test_n_plan0)
            test_n_move0=(r[testing_data[i]].unit[l].spikeTimes>p2) & (r[testing_data[i]].unit[l].spikeTimes<p3)
            test_n_move1=np.sum(test_n_move0)
        elif list(r[testing_data[i]].unit[l].spikeTimes) == []:
            test_N1=0
            test_n_plan1=0
            test_n_move1=0
        else:
            test_N0=(r[testing_data[i]].unit[l].spikeTimes>p1) & (r[testing_data[i]].unit[l].spikeTimes<p3)
            test_N1=np.sum(test_N0)
            test_n_plan0=(r[testing_data[i]].unit[l].spikeTimes>p1) & (r[testing_data[i]].unit[l].spikeTimes<p2)
            test_n_plan1=np.sum(test_n_plan0)
            test_n_move0=(r[testing_data[i]].unit[l].spikeTimes>p2) & (r[testing_data[i]].unit[l].spikeTimes<p3)
            test_n_move1=np.sum(test_n_move0)
        test_N_time1=p3-p1
        test_n_movetime1=p3-p2
        test_n_plantime1=p2-p1
        
        test_N2=np.append(test_N2,test_N1)
        
        test_n_plan2=np.append(test_n_plan2,test_n_plan1)
        
        test_n_move2=np.append(test_n_move2,test_n_move1)
        
    test_N.append(test_N2)
    test_N_time.append(test_N_time1)
    test_n_plan.append(test_n_plan2)
    test_n_plantime.append(test_n_plantime1)
    test_n_move.append(test_n_move2)
    test_n_movetime.append(test_n_movetime1)

In [10]:
test_target0=[cfr[i] for i in testing_data]
test_table1=pd.DataFrame(test_target0,index=testing_data,columns=['targets']) # index represent the i th trials
test_table1['Combined']=test_N
test_table1['Combined_time']=test_N_time
test_table1['n_plan']=test_n_plan
test_table1['n_plantime']=test_n_plantime
test_table1['n_move']=test_n_move
test_table1['n_movetime']=test_n_movetime

test_table1['Combined_rate']=test_table1['Combined']/test_table1['Combined_time']
test_table1['n_plan_rate']=test_table1['n_plan']/test_table1['n_plantime']
test_table1['n_move_rate']=test_table1['n_move']/test_table1['n_movetime']

test_table1

,targets,Combined,Combined_time,n_plan,n_plantime,n_move,n_movetime,Combined_rate,n_plan_rate,n_move_rate
1,1,"[1.0, 11.0, 1.0, 16.0, 16.0, 11.0, 7.0, 18.0, ...",1525,"[1.0, 6.0, 0.0, 7.0, 7.0, 3.0, 1.0, 10.0, 4.0,...",1005,"[0.0, 5.0, 1.0, 9.0, 9.0, 8.0, 6.0, 8.0, 4.0, ...",520,"[0.000655737704918, 0.0072131147541, 0.0006557...","[0.000995024875622, 0.00597014925373, 0.0, 0.0...","[0.0, 0.00961538461538, 0.00192307692308, 0.01..."
3,5,"[0.0, 17.0, 3.0, 14.0, 25.0, 12.0, 23.0, 13.0,...",1540,"[0.0, 9.0, 2.0, 6.0, 11.0, 5.0, 10.0, 6.0, 3.0...",1005,"[0.0, 8.0, 1.0, 8.0, 14.0, 7.0, 13.0, 7.0, 8.0...",535,"[0.0, 0.011038961039, 0.00194805194805, 0.0090...","[0.0, 0.0089552238806, 0.00199004975124, 0.005...","[0.0, 0.014953271028, 0.0018691588785, 0.01495..."
4,6,"[0.0, 7.0, 0.0, 28.0, 27.0, 15.0, 10.0, 20.0, ...",1250,"[0.0, 3.0, 0.0, 15.0, 9.0, 9.0, 6.0, 7.0, 5.0,...",755,"[0.0, 4.0, 0.0, 13.0, 18.0, 6.0, 4.0, 13.0, 10...",495,"[0.0, 0.0056, 0.0, 0.0224, 0.0216, 0.012, 0.00...","[0.0, 0.00397350993377, 0.0, 0.0198675496689, ...","[0.0, 0.00808080808081, 0.0, 0.0262626262626, ..."
5,8,"[0.0, 16.0, 1.0, 16.0, 20.0, 20.0, 15.0, 18.0,...",1500,"[0.0, 8.0, 0.0, 3.0, 10.0, 12.0, 8.0, 8.0, 3.0...",1005,"[0.0, 8.0, 1.0, 13.0, 10.0, 8.0, 7.0, 10.0, 4....",495,"[0.0, 0.0106666666667, 0.000666666666667, 0.01...","[0.0, 0.00796019900498, 0.0, 0.00298507462687,...","[0.0, 0.0161616161616, 0.0020202020202, 0.0262..."
6,8,"[0.0, 15.0, 0.0, 22.0, 25.0, 16.0, 13.0, 13.0,...",1275,"[0.0, 6.0, 0.0, 11.0, 8.0, 6.0, 7.0, 4.0, 2.0,...",755,"[0.0, 9.0, 0.0, 11.0, 17.0, 10.0, 6.0, 9.0, 12...",520,"[0.0, 0.0117647058824, 0.0, 0.0172549019608, 0...","[0.0, 0.00794701986755, 0.0, 0.0145695364238, ...","[0.0, 0.0173076923077, 0.0, 0.0211538461538, 0..."
7,4,"[0.0, 13.0, 5.0, 14.0, 24.0, 16.0, 8.0, 10.0, ...",1275,"[0.0, 3.0, 2.0, 4.0, 12.0, 5.0, 1.0, 7.0, 5.0,...",755,"[0.0, 10.0, 3.0, 10.0, 12.0, 11.0, 7.0, 3.0, 1...",520,"[0.0, 0.0101960784314, 0.00392156862745, 0.010...","[0.0, 0.00397350993377, 0.00264900662252, 0.00...","[0.0, 0.0192307692308, 0.00576923076923, 0.019..."
8,7,"[0.0, 5.0, 0.0, 15.0, 17.0, 14.0, 11.0, 3.0, 9...",1255,"[0.0, 3.0, 0.0, 8.0, 7.0, 4.0, 4.0, 1.0, 1.0, ...",755,"[0.0, 2.0, 0.0, 7.0, 10.0, 10.0, 7.0, 2.0, 8.0...",500,"[0.0, 0.00398406374502, 0.0, 0.0119521912351, ...","[0.0, 0.00397350993377, 0.0, 0.0105960264901, ...","[0.0, 0.004, 0.0, 0.014, 0.02, 0.02, 0.014, 0...."
10,1,"[4.0, 15.0, 3.0, 21.0, 27.0, 10.0, 6.0, 23.0, ...",1260,"[4.0, 5.0, 0.0, 10.0, 14.0, 1.0, 0.0, 6.0, 1.0...",755,"[0.0, 10.0, 3.0, 11.0, 13.0, 9.0, 6.0, 17.0, 3...",505,"[0.0031746031746, 0.0119047619048, 0.002380952...","[0.00529801324503, 0.00662251655629, 0.0, 0.01...","[0.0, 0.019801980198, 0.00594059405941, 0.0217..."
12,7,"[0.0, 9.0, 2.0, 20.0, 17.0, 13.0, 18.0, 15.0, ...",1225,"[0.0, 2.0, 2.0, 7.0, 4.0, 10.0, 7.0, 7.0, 5.0,...",755,"[0.0, 7.0, 0.0, 13.0, 13.0, 3.0, 11.0, 8.0, 8....",470,"[0.0, 0.00734693877551, 0.00163265306122, 0.01...","[0.0, 0.00264900662252, 0.00264900662252, 0.00...","[0.0, 0.0148936170213, 0.0, 0.0276595744681, 0..."
14,3,"[0.0, 5.0, 1.0, 9.0, 18.0, 12.0, 7.0, 21.0, 15...",1335,"[0.0, 1.0, 1.0, 1.0, 6.0, 2.0, 0.0, 10.0, 5.0,...",755,"[0.0, 4.0, 0.0, 8.0, 12.0, 10.0, 7.0, 11.0, 10...",580,"[0.0, 0.00374531835206, 0.000749063670412, 0.0...","[0.0, 0.00132450331126, 0.00132450331126, 0.00...","[0.0, 0.00689655172414, 0.0, 0.0137931034483, ..."


In [11]:
from scipy.stats import multivariate_normal
def combined_simulate(r1):
    f=[]
    for l in range(8):
        
        f1=multivariate_normal.logpdf(r1, mean=combined_mean[l], cov=np.diag(np.diag(combined_cov[l])),allow_singular=True)
        f.append(f1)
    simulate_target=f.index(max(f))+1
    return simulate_target

In [12]:
combined_simulate_targets=[]
for i in range(len(test_table1)):
    r1=list(test_table1['Combined_rate'])[i]
    simulate_target=combined_simulate(r1)
    combined_simulate_targets.append(simulate_target)

In [13]:
orginal_degrees=[degrees[i-1] for i in test_table1['targets']]
combined_simulate_degrees=[degrees[i-1] for i in combined_simulate_targets]

combined_e=abs(np.array(orginal_degrees)-np.array(combined_simulate_degrees))
correct_combined=[i==j for i,j in zip(test_table1['targets'],combined_simulate_targets)]
correct_percent=sum(correct_combined)/len(test_table1['targets'])

print(np.mean(combined_e))
print(np.std(combined_e))
print(correct_percent)

1.26251106916
13.1157873503
0.977991746905


In [14]:
def plan_simulate(r1):
    f=[]
    for l in range(8):

        f1=multivariate_normal.logpdf(r1, mean=plan_mean[l], cov=np.diag(np.diag(plan_cov[l])),allow_singular=True)
        f.append(f1)
    simulate_target=f.index(max(f))+1
    return simulate_target

In [15]:
plan_simulate_targets=[]
for i in range(len(test_table1)):
    r1=list(test_table1['n_plan_rate'])[i]
    simulate_target=plan_simulate(r1)
    plan_simulate_targets.append(simulate_target)

In [16]:
plan_simulate_degrees=[degrees[i-1] for i in plan_simulate_targets]

plan_e=abs(np.array(orginal_degrees)-np.array(plan_simulate_degrees))
correct_plan=[i==j for i,j in zip(test_table1['targets'],plan_simulate_targets)]
plan_percent=sum(correct_plan)/len(test_table1['targets'])
print(np.mean(plan_e))
print(np.std(plan_e))
print(plan_percent)

16.8786112608
51.4964632152
0.760660247593


In [17]:
def move_simulate(r1):
    f=[]
    for l in range(8):
        f1=multivariate_normal.logpdf(r1, mean=move_mean[l], cov=np.diag(np.diag(move_cov[l])),allow_singular=True)
        f.append(f1)
    simulate_target=f.index(max(f))+1
    return simulate_target

In [18]:
move_simulate_targets=[]
for i in range(len(test_table1)):
    r1=list(test_table1['n_move_rate'])[i]
    simulate_target=move_simulate(r1)
    move_simulate_targets.append(simulate_target)

In [19]:
move_simulate_degrees=[degrees[i-1] for i in move_simulate_targets]

move_e=abs(np.array(orginal_degrees)-np.array(move_simulate_degrees))
correct_move=[i==j for i,j in zip(test_table1['targets'],move_simulate_targets)]
move_percent=sum(correct_move)/len(test_table1['targets'])
print(np.mean(move_e))
print(np.std(move_e))
print(move_percent)

4.06962858789
25.2801939692
0.936726272352


In [20]:
def P_M_rate_simulate(r1):
    f=[]
    for l in range(8):
        mean=np.append(plan_mean[l],move_mean[l])
        cov=np.diag(np.append(np.diag(plan_cov[l]),np.diag(move_cov[l])))
        f1=multivariate_normal.logpdf(r1, mean=mean, cov=cov,allow_singular=True)
        f.append(f1)
    simulate_target=f.index(max(f))+1
    return simulate_target

In [21]:
PMrate_simulate_targets=[]
for i in range(len(test_table1)):
    r1=np.append(np.array(list(test_table1['n_plan_rate'])[i]),np.array(list(test_table1['n_move_rate'])[i]))
    simulate_target=P_M_rate_simulate(r1)
    PMrate_simulate_targets.append(simulate_target)

In [22]:
PMrate_simulate_degrees=[degrees[i-1] for i in PMrate_simulate_targets]

PMrate_e=abs(np.array(orginal_degrees)-np.array(PMrate_simulate_degrees))
correct_PMrate=[i==j for i,j in zip(test_table1['targets'],PMrate_simulate_targets)]
PMrate_percent=sum(correct_PMrate)/len(test_table1['targets'])
print(np.mean(PMrate_e))
print(np.std(PMrate_e))
print(PMrate_percent)

4.61820413636
27.9183671974
0.932599724897


# PC score

In [23]:
def pc_projection(X):
    mu = np.mean(X,axis=0) # calculate mean
    w,v = np.linalg.eig(np.cov(X.T)) # calculate eigenvalues of covariance matrix
    scores = np.dot((X - mu),v[:,0]) # project into lower dimensional space
    return scores

In [24]:
from scipy import ndimage
plan_pc=[]
move_pc=[]
for i in range(len(training_data)):
    plan_pc1=[]
    move_pc1=[]
    p1=r[training_data[i]].timeTouchHeld
    p2=r[training_data[i]].timeGoCue
    p3=r[training_data[i]].timeTargetAcquire
    plan_series=np.linspace(p1,p2,(p2-p1)/5+1)
    move_series=np.linspace(p2,p3,(p3-p2)/5+1)

    for l in range(190):
        plan_bin=np.zeros(len(plan_series))
        move_bin=np.zeros(len(move_series))
        if type(r[training_data[i]].unit[l].spikeTimes) == float:   

            if (r[training_data[i]].unit[l].spikeTimes>=p1) & (r[training_data[i]].unit[l].spikeTimes<p2):
                id_plan=math.floor((r[training_data[i]].unit[l].spikeTimes-p1)/5)
                plan_bin[id_plan] += 1
            if (r[training_data[i]].unit[l].spikeTimes>=p2) & (r[training_data[i]].unit[l].spikeTimes<p3):
                id_move=math.floor((r[training_data[i]].unit[l].spikeTimes-p2)/5)
                move_bin[id_move] += 1
        elif list(r[training_data[i]].unit[l].spikeTimes) == []:  
            pass
        else:                                                     
            for m in r[training_data[i]].unit[l].spikeTimes:
                if (m>=p1) & (m<p2):
                    id_plan=math.floor((m-p1)/5)
                    plan_bin[id_plan] += 1
                if (m>=p2) & (m<p3):
                    id_move=math.floor((m-p2)/5)
                    move_bin[id_move] += 1
        plan_bin=plan_bin/5
        move_bin=move_bin/5
        plan_convolve=ndimage.filters.gaussian_filter(plan_bin,sigma=30,truncate=5)
        move_convolve=ndimage.filters.gaussian_filter(move_bin,sigma=30,truncate=5)
        plan_pc1.append(plan_convolve)
        move_pc1.append(move_convolve)
    plan_pc1=np.array(plan_pc1)
    move_pc1=np.array(move_pc1)
    plan_pcscore=abs(pc_projection(plan_pc1))
    move_pcscore=abs(pc_projection(move_pc1))
    plan_pc.append(plan_pcscore)
    move_pc.append(move_pcscore)

In [25]:
target0=[cfr[i] for i in training_data]
table_pc=pd.DataFrame(target0,index=training_data,columns=['targets']) # index represent the i th trials
table_pc['plan_pc']=plan_pc
table_pc['move_pc']=move_pc
table_pc

,targets,plan_pc,move_pc
0,4,"[0.0562180852119, 0.0151906689296, 0.082038637...","[0.172352861001, 0.0172840841421, 0.1095171838..."
2,7,"[0.103787325486, 0.0728671130197, 0.0870227952...","[0.171094754017, 0.00435224588755, 0.149040286..."
9,1,"[0.0300549527368, 0.00124694646698, 0.08624950...","[0.142354539508, 0.0172913639212, 0.1281076379..."
11,5,"[0.0457900597661, 0.02986018004, 0.09405476009...","[0.203007308422, 0.00306609108449, 0.203007308..."
13,2,"[0.113868329997, 0.0422405756976, 0.0570542756...","[0.15771569648, 0.105126234053, 0.124414365016..."
16,7,"[0.122148906835, 0.0576395048072, 0.0794461055...","[0.158179110818, 0.0411714572342, 0.1365540443..."
17,4,"[0.0693804713783, 0.051085269952, 0.0493302921...","[0.170202627943, 0.013852971619, 0.12948203759..."
23,6,"[0.100881912013, 0.0568881904169, 0.1008819120...","[0.160667503163, 0.123582716242, 0.16066750316..."
24,8,"[0.0993950684382, 0.0324644232283, 0.099395068...","[0.151750598792, 0.0189520660544, 0.1282788810..."
25,7,"[0.129304705477, 0.0293184517799, 0.1013739038...","[0.174343054497, 0.0331435924153, 0.1327529075..."


In [26]:
plan_pc_mean=[]
plan_pc_cov=[]

move_pc_mean=[]
move_pc_cov=[]

for i in range(8):
    i=i+1

    plan_pc=np.array(list(table_pc[table_pc.targets==i]['plan_pc']))
    plan_pc_mean1=np.mean(plan_pc,axis=0)

        
    plan_pc_mean.append(plan_pc_mean1)
    plan_pc_cov.append(np.cov(plan_pc.T))
    
    move_pc=np.array(list(table_pc[table_pc.targets==i]['move_pc']))
    move_pc_mean1=np.mean(move_pc,axis=0)

        
    move_pc_mean.append(move_pc_mean1)
    move_pc_cov.append(np.cov(move_pc.T))

In [27]:
test_plan_pc=[]
test_move_pc=[]
for i in range(len(testing_data)):
    test_plan_pc1=[]
    test_move_pc1=[]
    p1=r[testing_data[i]].timeTouchHeld
    p2=r[testing_data[i]].timeGoCue
    p3=r[testing_data[i]].timeTargetAcquire
    test_plan_series=np.linspace(p1,p2,(p2-p1)/5+1)
    test_move_series=np.linspace(p2,p3,(p3-p2)/5+1)

    for l in range(190):
        test_plan_bin=np.zeros(len(test_plan_series))
        test_move_bin=np.zeros(len(test_move_series))
        if type(r[testing_data[i]].unit[l].spikeTimes) == float:   # when there is only one spike and its spiketime

            if (r[testing_data[i]].unit[l].spikeTimes>=p1) & (r[testing_data[i]].unit[l].spikeTimes<p2):
                test_id_plan=math.floor((r[testing_data[i]].unit[l].spikeTimes-p1)/5)
                test_plan_bin[test_id_plan] += 1
            if (r[testing_data[i]].unit[l].spikeTimes>=p2) & (r[testing_data[i]].unit[l].spikeTimes<p3):
                test_id_move=math.floor((r[testing_data[i]].unit[l].spikeTimes-p2)/5)
                test_move_bin[test_id_move] += 1
        elif list(r[testing_data[i]].unit[l].spikeTimes) == []:   # when there is no spike and its spiketime
            pass
        else:                                                     # when there are more than one spikes and their spiketimes
            for m in r[testing_data[i]].unit[l].spikeTimes:
                if (m>=p1) & (m<p2):
                    test_id_plan=math.floor((m-p1)/5)
                    test_plan_bin[test_id_plan] += 1
                if (m>=p2) & (m<p3):
                    test_id_move=math.floor((m-p2)/5)
                    test_move_bin[test_id_move] += 1
        test_plan_bin=test_plan_bin/5
        test_move_bin=test_move_bin/5
        test_plan_convolve=ndimage.filters.gaussian_filter(test_plan_bin,sigma=10,truncate=5)
        test_move_convolve=ndimage.filters.gaussian_filter(test_move_bin,sigma=10,truncate=5)
        test_plan_pc1.append(test_plan_convolve)
        test_move_pc1.append(test_move_convolve)
    test_plan_pc1=np.array(test_plan_pc1)
    test_move_pc1=np.array(test_move_pc1)
    test_plan_pc.append(abs(pc_projection(test_plan_pc1)))
    test_move_pc.append(abs(pc_projection(test_move_pc1)))

In [28]:
target0=[cfr[i] for i in testing_data]
test_table_pc=pd.DataFrame(target0,index=testing_data,columns=['targets']) # index represent the i th trials
test_table_pc['plan_pc']=test_plan_pc
test_table_pc['move_pc']=test_move_pc
test_table_pc

,targets,plan_pc,move_pc
1,1,"[0.0975317573501, 0.0173389159678, 0.112426647...","[0.160771324128, 0.0507086168173, 0.1384710634..."
3,5,"[0.12072604848, 0.0141809357051, 0.09439713209...","[0.177388906627, 0.0297343653312, 0.1625802477..."
4,6,"[0.10438046737, 0.0498629165125, 0.10438046737...","[0.163930291307, 0.0729262282634, 0.1639302913..."
5,8,"[0.113046153112, 0.00976509427422, 0.113046153...","[0.157256982365, 0.00131274361736, 0.145817404..."
6,8,"[0.112665935262, 0.0152859983738, 0.1126659352...","[0.162315983408, 0.00526116700411, 0.162315983..."
7,4,"[0.0854110733207, 0.0328979000875, 0.048471159...","[0.171498381915, 0.0167015771149, 0.1103267505..."
8,7,"[0.0820180505433, 0.0286359593, 0.082018050543...","[0.150877644484, 0.110555283274, 0.15087764448..."
10,1,"[0.0498036434909, 0.0234840783257, 0.109007963...","[0.160992232408, 0.0543034299456, 0.1163754459..."
12,7,"[0.102295072848, 0.0675164848757, 0.0680097389...","[0.16375058941, 0.0445192516216, 0.16375058941..."
14,3,"[0.10847036639, 0.091833390555, 0.089800013255...","[0.159242535631, 0.0729134054885, 0.1592425356..."


## Plan PC and Move PC

In [29]:
def P_M_pcscore_simulate(r1):
    f=[]
    for l in range(8):
        mean=np.append(plan_pc_mean[l],move_pc_mean[l])
        cov=np.diag(np.append(np.diag(plan_pc_cov[l]),np.diag(move_pc_cov[l])))
        f1=multivariate_normal.logpdf(r1, mean=mean, cov=cov,allow_singular=True)
        f.append(f1)
    simulate_target=f.index(max(f))+1
    return simulate_target

In [30]:
PMpcscore_simulate_targets=[]
for i in range(len(test_table_pc)):
    r1=np.append(list(test_table_pc['plan_pc'])[i],list(test_table_pc['move_pc'])[i])
    simulate_target=P_M_pcscore_simulate(r1)
    PMpcscore_simulate_targets.append(simulate_target)

In [31]:
PMpcscore_simulate_degrees=[degrees[i-1] for i in PMpcscore_simulate_targets]

PMpcscore_e=abs(np.array(orginal_degrees)-np.array(PMpcscore_simulate_degrees))
correct_PMpcscore=[i==j for i,j in zip(test_table_pc['targets'],PMpcscore_simulate_targets)]
PMpcscore_percent=sum(correct_PMpcscore)/len(test_table_pc['targets'])
print(np.mean(PMpcscore_e))
print(np.std(PMpcscore_e))
print(PMpcscore_percent)

3.57456303345
18.9506920204
0.931224209078


## Plan rate and Move PC

In [32]:
def Prate_Mpc_simulate(r1):
    f=[]
    for l in range(8):
        mean=np.append(plan_mean[l],move_pc_mean[l])
        cov=np.diag(np.append(np.diag(plan_cov[l]),np.diag(move_pc_cov[l])))
        f1=multivariate_normal.logpdf(r1, mean=mean, cov=cov,allow_singular=True)
        f.append(f1)
    simulate_target=f.index(max(f))+1
    return simulate_target

In [33]:
Prate_Mpc_simulate_targets=[]
for i in range(len(test_table_pc)):
    r1=np.append(list(test_table1['n_plan_rate'])[i],list(test_table_pc['move_pc'])[i])
    simulate_target=Prate_Mpc_simulate(r1)
    Prate_Mpc_simulate_targets.append(simulate_target)

In [34]:
Prate_Mpc_simulate_degrees=[degrees[i-1] for i in Prate_Mpc_simulate_targets]

Prate_Mpc_e=abs(np.array(orginal_degrees)-np.array(Prate_Mpc_simulate_degrees))
correct_Prate_Mpc=[i==j for i,j in zip(test_table_pc['targets'],Prate_Mpc_simulate_targets)]
Prate_Mpc_percent=sum(correct_Prate_Mpc)/len(test_table_pc['targets'])
print(np.mean(Prate_Mpc_e))
print(np.std(Prate_Mpc_e))
print(Prate_Mpc_percent)

4.23472795846
20.9146159643
0.920220082531
